In [5]:
import pandas as pd
from datetime import datetime

IN_PATH = "spl_full_data.csv"      # change to your bronze file
OUT_PATH = "data_for_model.csv"

df = pd.read_csv(IN_PATH)
df.head()


,season_id,season_label,event_id,home_team,away_team,fetch_ok,match_url,match_date,match_time,ok,...,home_tackles_won,home_tackles_won_total,home_through_balls,home_throw_ins,home_total_saves,home_total_shots,home_total_shots__totalshotsongoal,home_total_tackles,home_touches_in_penalty_area,home_yellow_cards
0,48772,2023,11041308,Tanjong Pagar Utd.,Albirex Niigata,1,https://www.sofascore.com/football/match/albir...,11/08/2023,07:45,1,...,NaN,NaN,0.0,17.0,2.0,13.0,13.0,1.0,14.0,2.0
1,48772,2023,11041309,Albirex Niigata,Lion City Sailors,1,https://www.sofascore.com/football/match/albir...,16/07/2023,06:00,1,...,NaN,NaN,4.0,27.0,1.0,17.0,17.0,0.0,28.0,0.0
2,48772,2023,11041310,Balestier Khalsa,Tampines Rovers,1,https://www.sofascore.com/football/match/tampi...,26/08/2023,06:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48772,2023,11041311,Geylang Int.,Young Lions,1,https://www.sofascore.com/football/match/young...,19/08/2023,06:00,1,...,NaN,NaN,2.0,19.0,7.0,17.0,17.0,0.0,22.0,0.0
4,48772,2023,11041312,Hougang Utd,DPMM,1,https://www.sofascore.com/football/match/brune...,04/08/2023,07:45,1,...,NaN,NaN,1.0,24.0,4.0,17.0,17.0,3.0,25.0,0.0


In [6]:
df["match_date_parsed"] = pd.to_datetime(
    df["match_date"],
    errors="coerce",
    dayfirst=True
)

df[["match_date", "match_date_parsed"]].head()


,match_date,match_date_parsed
0,11/08/2023,2023-08-11
1,16/07/2023,2023-07-16
2,26/08/2023,2023-08-26
3,19/08/2023,2023-08-19
4,04/08/2023,2023-08-04


In [7]:
today = pd.Timestamp(datetime.now().date())

df["is_future_fixture"] = (df["match_date_parsed"] > today).astype(int)

df[["match_date_parsed", "is_future_fixture"]].head()


,match_date_parsed,is_future_fixture
0,2023-08-11,0
1,2023-07-16,0
2,2023-08-26,0
3,2023-08-19,0
4,2023-08-04,0


In [8]:
silver = df[[
    "season_id",
    "season_label",
    "event_id",
    "home_team",
    "away_team",
    "match_url",
    "match_date_parsed",
    "home_expected_goals",
    "away_expected_goals",
    "is_future_fixture"
]].copy()


In [9]:
silver = silver.rename(columns={
    "match_date_parsed": "match_date",
    "expected_goals_home": "home_expected_goals",
    "expected_goals_away": "away_expected_goals"
})

silver["match_date"] = silver["match_date"].dt.strftime("%Y-%m-%d")

silver.head()


,season_id,season_label,event_id,home_team,away_team,match_url,match_date,home_expected_goals,away_expected_goals,is_future_fixture
0,48772,2023,11041308,Tanjong Pagar Utd.,Albirex Niigata,https://www.sofascore.com/football/match/albir...,2023-08-11,1.37,3.40,0
1,48772,2023,11041309,Albirex Niigata,Lion City Sailors,https://www.sofascore.com/football/match/albir...,2023-07-16,2.20,1.37,0
2,48772,2023,11041310,Balestier Khalsa,Tampines Rovers,https://www.sofascore.com/football/match/tampi...,2023-08-26,NaN,NaN,0
3,48772,2023,11041311,Geylang Int.,Young Lions,https://www.sofascore.com/football/match/young...,2023-08-19,2.17,0.72,0
4,48772,2023,11041312,Hougang Utd,DPMM,https://www.sofascore.com/football/match/brune...,2023-08-04,1.37,0.53,0


In [10]:
silver.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


Saved: data_for_model.csv
